In [ ]:
! pip install pycaret[full]
! pip install markupsafe==2.0.1

# 1. 모델링



## 1) CatBoost

In [ ]:
! pip install catboost
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 120 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 209 kB 46.8 MB/s 
     |████████████████████████████████| 81 kB 10.2 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 147 kB 59.6 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 112 kB 63.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=87449115be2159afc83655c7d264d4c2aa2175eff67bc33bbba856d4dd7a59e4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
import random
random.seed(42)
np.random.seed(42)

# 모델 평가 지표 
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col
    
submit = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/2-2_검증데이터셋.csv', encoding='cp949')
hospital = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/hospital.csv', encoding='cp949')
new_merge_data = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/new_merge_data.csv', encoding='cp949')
forecast = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/예보지수.csv', encoding='cp949')

In [ ]:
new_merge_data = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/new_merge_data.csv', encoding='cp949')
# 지연효과 반영 
def add_lag_feature(dat, lag = None):

    cols = ['sum_ss_hr_max', 'ssrate_max', 'avg_rhm_max', 'min_rhm_max', 'sum_rn_max', 
            'max_ws_max','avg_ws_max','max_ta_max', 'min_ta_max', 'PM10_max'] # 'avg_tca_max', 
    '''
    cols = ['avg_tca_mean','sum_ss_hr_mean', 'ssrate_mean','avg_rhm_mean', 'min_rhm_mean','sum_rn_mean', 
            'avg_ws_mean', 'max_ws_mean', 'avg_ta_mean', 'max_ta_mean', 'min_ta_mean', 'PM10_mean']
            
    cols = ['avg_tca_mean', 'avg_tca_max','sum_ss_hr_mean','sum_ss_hr_max', 'ssrate_mean', 
            'ssrate_max', 'avg_rhm_mean','avg_rhm_max', 'min_rhm_mean', 'min_rhm_max', 
            'sum_rn_mean', 'sum_rn_max', 'avg_ws_mean', 'avg_ws_max', 'max_ws_mean', 
            'max_ws_max','avg_ta_mean', 'avg_ta_max','max_ta_mean', 'max_ta_max', 
            'min_ta_mean', 'min_ta_max','PM10_mean','PM10_max']
    '''
    group_df = dat.groupby(['area'])
    lag_mean = group_df[cols].rolling(window = lag, min_periods=0, closed='left').mean().reset_index(0,drop=True)
    for col in cols:
        dat[f'{col}/mean_lag{lag}'] = lag_mean[col]
    return dat

add_lag_feature(new_merge_data, 3)

final = pd.merge(hospital, new_merge_data, how='left')
final = pd.merge(final, forecast, how='left')
final = final.sort_values(['yyyymmdd', 'sex', 'area']).reset_index(drop=True)
# 지금까진 2,4가 best, 1.1888
# 3,5 + closd = 'left(3 <= X < 6) => 0.18926

# 새로운 변수 생성 
dates = pd.to_datetime(final['yyyymmdd'], format='%Y%m%d').dt      
final['weekday']= dates.weekday                                     # 요일 
final['risk_ratio'] = final['num_risk_age'] / final['tot_person']   # 위험인구 비율 
final['day_diff'] = final['max_ta_mean'] - final['min_ta_mean']     # 일교차 
final['temp_humi'] = final['avg_rhm_mean'] * final['avg_ta_mean']   # 습도 * 기온
# https://www.karger.com/Article/Fulltext/505122

final['temp_threshold'] = np.where((final['min_ta_mean'] < -1.5), 1, 0)  # 뇌졸중 영향 온도 threshhold
# https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0094070

conditionlist = [
    (final['month'].isin([3,4,5])),
    (final['month'].isin([6,7,8])),
    (final['month'].isin([9,10,11])),
    (final['month'].isin([12,1,2]))]
choicelist = [1,2,3,4]
final['season'] = np.select(conditionlist, choicelist, default='Not Specified')  # 계절 
final['diff_risk'] = np.where(final['day_diff'] >= 10, 1, 0)                     # 일교차 10도 이상 

# 월 평균 환자 수 의 평균값보다 많은면 risk_month = 1 ,아니면 risk_month = 0
final_cpy = final.copy()
final_cpy = final_cpy.groupby(['month'])[final_cpy.columns].mean()
mean_sick = final_cpy['frequency'].mean()
for year in range(2012,2016+1):
  for month in range(1,12+1):
    if (final.loc[(final['year']==year) & (final['month']==month),'frequency'].mean()) >= mean_sick:
      final.loc[(final['year']==year) & (final['month']==month),'risk_month'] = 1
    else:
      final.loc[(final['year']==year) & (final['month']==month),'risk_month'] = 0

del final_cpy

final = final.drop(['avg_tca_mean', 'avg_tca_max','num_risk_age', 
                    'sum_rn_mean', 'sum_ss_hr_mean',
                    # 'sum_ss_hr_max', 
                    # 'ssrate_mean',
                    # 'ssrate_max', 
                    # 'avg_rhm_mean', 
                    # 'avg_rhm_max', 
                    # 'min_rhm_mean',
                    # 'min_rhm_max', 
                    # 'sum_rn_max', 
                    # 'avg_ws_mean', 
                    # 'avg_ws_max', 
                    # 'max_ws_mean',
                    # 'max_ws_max', 
                    # 'avg_ta_mean', 
                    # 'avg_ta_max', 
                    # 'max_ta_mean', 
                    # 'max_ta_max',
                    # 'min_ta_mean', 
                    # 'min_ta_max',

                    'SO2_mean', 'SO2_max', 
                    'CO_mean', 
                    # 'CO_max', 
                    # 'O3_mean', 
                    'O3_max', 
                    # 'NO2_mean',
                    # 'NO2_max',
                    'A03_t1_max', 'A03_t2_max', 'A03_t3_max',
                    'A06_t1_max', 'A06_t2_max', 'A06_t3_max',
                    'A07_t1_max',  'A07_t2_max','A07_t3_max',


                     'A03_t1_mean', 'A03_t2_mean', 'A03_t3_mean', 
                     'A04_t1_mean', 'A04_t2_mean', 'A04_t3_mean',
                     'A04_t1_max' ,'A04_t2_max','A04_t3_max',
                    # 'A05_t1_mean', 'A05_t2_mean','A05_t3_mean',
                     'A05_t1_max','A05_t2_max','A05_t3_max',
                    # 'A06_t1_mean', 'A06_t2_mean', 'A06_t3_mean',
                    # 'A07_t1_mean', 'A07_t2_mean', 'A07_t3_mean', 
                    ], axis=1) # 5,6,7의 mean만 쓴게 1.191


# train, test split
train = final[final['year'] != 2016].drop('yyyymmdd', axis=1)
test = final[final['year'] == 2016].drop(['frequency', 'yyyymmdd'], axis=1)

X = train.drop('frequency', axis=1)
y = train['frequency']

# X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42, stratify=X['month'])

# # 모델링 
# CAT = CatBoostRegressor(verbose=2, random_state=42, one_hot_max_size = 17)
# CAT.fit(X_train, y_train,
#         cat_features=['area', 'sex', 'year', 'month', 'day', 'day_differ', 
#                       'month_differ', 'weekday', 'temp_threshold', 'season', 'diff_risk'],
#         eval_set=(X_val, y_val),
#         early_stopping_rounds = 200, use_best_model = True)

In [ ]:
# 학습함수 정의 
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42, stratify=X['month'])
def objective(trial: Trial):
    param = {
        'cat_features' : ['area', 'sex', 'year', 'month', 'day', 'day_differ', 
                          'month_differ', 'weekday', 'temp_threshold', 'season', 'diff_risk'],
        'one_hot_max_size' : 17,
        'loss_function': 'RMSE',
        'random_state' : 42,
        'n_estimators':  10000,
        'early_stopping_rounds' : 100,
        'max_depth': trial.suggest_int('max_depth', 3, 16),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3, step=0.05),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 1.5, step=0.05),

         'max_bin': trial.suggest_int('max_bin', 200, 1000, step=50),   
        # 'rsm': trial.suggest_float('rsm', 0.3, 1.0, step=0.1),
        # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 300, step=10),
        # 'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15)
    }
    model = CatBoostRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_val,y_val)])
    rmse = RMSE(y_val, model.predict(X_val))

    return rmse

# create study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='CatBoostRegressor-Hyperparameter-Tuning'
)

# 학습 
CAT_study = study.optimize(lambda trial: objective(trial), n_trials=100)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/2-2_검증데이터셋.csv', encoding='cp949')    
pred = CAT.predict(test)

year_frequency = final.groupby(['year'])['frequency'].mean()
rate = year_frequency.values[3] / year_frequency.values[2]

submit['frequency'] = pred * rate
submit.to_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/220139.csv', encoding='cp949', index=False)

In [ ]:
from tqdm import tqdm
CAT = CatBoostRegressor(verbose=1, random_seed=42, cat_features=['area', 'sex', 'year', 'month', 'day', 'day_differ', 
                      'month_differ', 'weekday', 'temp_threshold', 'season', 'diff_risk'], one_hot_max_size = 17)
# K-Fold 
def kfold(model, tr):
    cv_accuracy = []
    cv = KFold(n_splits = 5, shuffle=True, random_state=42)
    n_iter = 0

    for t,v in tqdm(cv.split(tr, tr['frequency'])): 
        train_cv = tr.iloc[t]
        val_cv = tr.iloc[v]

        X = train_cv.drop('frequency', axis=1)
        y = train_cv['frequency']

        val_X = val_cv.drop('frequency', axis=1)
        val_y = val_cv['frequency']

        model.fit(X, y, eval_set = [(val_X, val_y)],
                  early_stopping_rounds=200, verbose=False, use_best_model = True)
        score = RMSE(val_y, model.predict(val_X))

        cv_accuracy.append(score)
        n_iter += 1
    
    return np.mean(cv_accuracy)
kfold(CAT, train)

5it [00:28,  5.67s/it]


1.212420107139476

### predict 10개 평균내기

In [ ]:
# 최종 예측치 생성 ==> 그냥 full로 예측하고 끝내는 거랑 이거랑 비교해서 제출(stacking이 아닌 단순 집계로 앙상블 할 경우의 얘기)
#CAT = CatBoostRegressor(**study.Best_trial)
year_frequency = final.groupby(['year'])['frequency'].mean()
rate = year_frequency.values[3] / year_frequency.values[2]
CAT = CatBoostRegressor(verbose=1, random_seed=42, one_hot_max_size = 17, 
                  cat_features=['area', 'sex', 'year', 'month', 'day', 'day_differ', 'month_differ', 'weekday', 'temp_threshold', 'season', 'diff_risk'])
def kfold_predict(model, train, test, num):
    preds = np.zeros(test.shape[0])
    cv = KFold(n_splits = num, shuffle=True, random_state=42)
    n_iter = 0
    rmse = []

    for t,v in cv.split(train, train['frequency']):#, groups=train['month']
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('frequency', axis=1)
        y = train_cv['frequency']

        val_X = val_cv.drop('frequency', axis=1)
        val_y = val_cv['frequency']
    
        model.fit(X, y, eval_set = [(val_X, val_y)],
                  early_stopping_rounds=200, verbose=False, use_best_model = True)
        
        preds += model.predict(test) / cv.n_splits

        rmse.append(RMSE(val_y, model.predict(val_X)))
        print(f'fold: {n_iter+1} ======> rmse : {rmse[n_iter]}')
        n_iter += 1
    print(f'total_mean ======> rmse : {np.mean(rmse)}')
    
    return preds
predict = kfold_predict(CAT, train, test, 10)

fold: 1 ======> rmse : 1.2202396009543792
fold: 2 ======> rmse : 1.2058785720797434
fold: 3 ======> rmse : 1.202290332226036
fold: 4 ======> rmse : 1.2308252962158626
fold: 5 ======> rmse : 1.217159928981238
fold: 6 ======> rmse : 1.228279884483892
fold: 7 ======> rmse : 1.2113453186133412
fold: 8 ======> rmse : 1.1921122450975177
fold: 9 ======> rmse : 1.2125627913066872
fold: 10 ======> rmse : 1.1969912400680915
total_mean ======> rmse : 1.211768521002679


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/2-2_검증데이터셋.csv', encoding='cp949')    
submit['frequency'] = predict * rate
submit.to_csv('/content/drive/MyDrive/ML_projects/weather/220139.csv', encoding='cp949', index=False)

## 2) LGBM

In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
import random
random.seed(42)
np.random.seed(42)
# 모델 평가 지표 
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col
    
submit = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/2-2_검증데이터셋.csv', encoding='cp949')
hospital = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/hospital.csv', encoding='cp949')
new_merge_data = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/new_merge_data.csv', encoding='cp949')
forecast = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/예보지수.csv', encoding='cp949')

In [ ]:
new_merge_data = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/new_merge_data.csv', encoding='cp949')
# 지연효과 반영 
def add_lag_feature(dat, lag = None):

    cols = ['sum_ss_hr_max', 'ssrate_max', 'avg_rhm_max', 'min_rhm_max', 'sum_rn_max', 
            'max_ws_max','avg_ws_max','max_ta_max', 'min_ta_max', 'PM10_max'] # 'avg_tca_max', 
    '''
    cols = ['avg_tca_mean','sum_ss_hr_mean', 'ssrate_mean','avg_rhm_mean', 'min_rhm_mean','sum_rn_mean', 
            'avg_ws_mean', 'max_ws_mean', 'avg_ta_mean', 'max_ta_mean', 'min_ta_mean', 'PM10_mean']
            
    cols = ['avg_tca_mean', 'avg_tca_max','sum_ss_hr_mean','sum_ss_hr_max', 'ssrate_mean', 
            'ssrate_max', 'avg_rhm_mean','avg_rhm_max', 'min_rhm_mean', 'min_rhm_max', 
            'sum_rn_mean', 'sum_rn_max', 'avg_ws_mean', 'avg_ws_max', 'max_ws_mean', 
            'max_ws_max','avg_ta_mean', 'avg_ta_max','max_ta_mean', 'max_ta_max', 
            'min_ta_mean', 'min_ta_max','PM10_mean','PM10_max']
    '''
    group_df = dat.groupby(['area'])
    lag_mean = group_df[cols].rolling(window = lag, min_periods=0, closed='left').mean().reset_index(0,drop=True)
    for col in cols:
        dat[f'{col}/mean_lag{lag}'] = lag_mean[col]
    return dat

add_lag_feature(new_merge_data, 3)

final = pd.merge(hospital, new_merge_data, how='left')
final = pd.merge(final, forecast, how='left')
final = final.sort_values(['yyyymmdd', 'sex', 'area']).reset_index(drop=True)
# 지금까진 2,4가 best, 1.1888
# 3,5 + closd = 'left(3 <= X < 6) => 0.18926

# 새로운 변수 생성 
dates = pd.to_datetime(final['yyyymmdd'], format='%Y%m%d').dt      
final['weekday']= dates.weekday                                     # 요일 
final['risk_ratio'] = final['num_risk_age'] / final['tot_person']   # 위험인구 비율 
final['day_diff'] = final['max_ta_mean'] - final['min_ta_mean']     # 일교차 
final['temp_humi'] = final['avg_rhm_mean'] * final['avg_ta_mean']   # 습도 * 기온
# https://www.karger.com/Article/Fulltext/505122

final['temp_threshold'] = np.where((final['min_ta_mean'] < -1.5), 1, 0)  # 뇌졸중 영향 온도 threshhold
# https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0094070

conditionlist = [
    (final['month'].isin([3,4,5])),
    (final['month'].isin([6,7,8])),
    (final['month'].isin([9,10,11])),
    (final['month'].isin([12,1,2]))]
choicelist = [1,2,3,4]
final['season'] = np.select(conditionlist, choicelist, default='Not Specified')  # 계절 
final['diff_risk'] = np.where(final['day_diff'] >= 10, 1, 0)                     # 일교차 10도 이상 

# 월 평균 환자 수 의 평균값보다 많은면 risk_month = 1 ,아니면 risk_month = 0
final_cpy = final.copy()
final_cpy = final_cpy.groupby(['month'])[final_cpy.columns].mean()
mean_sick = final_cpy['frequency'].mean()
for year in range(2012,2016+1):
  for month in range(1,12+1):
    if (final.loc[(final['year']==year) & (final['month']==month),'frequency'].mean()) >= mean_sick:
      final.loc[(final['year']==year) & (final['month']==month),'risk_month'] = 1
    else:
      final.loc[(final['year']==year) & (final['month']==month),'risk_month'] = 0

del final_cpy

final = final.drop(['avg_tca_mean', 'avg_tca_max','num_risk_age', 
                    'sum_rn_mean', 'sum_ss_hr_mean',
                    'SO2_mean', 'SO2_max', 
                    'CO_mean', 
                    # 'CO_max', 
                    # 'O3_mean', 
                    'O3_max', 
                    # 'NO2_mean',
                    # 'NO2_max',
                    'A03_t1_max', 'A03_t2_max', 'A03_t3_max',
                    'A06_t1_max', 'A06_t2_max', 'A06_t3_max',
                    'A07_t1_max',  'A07_t2_max','A07_t3_max',

                    # 'A03_t1_mean', 'A03_t2_mean', 'A03_t3_mean', 
                    # 'A04_t1_mean', 'A04_t2_mean', 'A04_t3_mean',
                    'A04_t1_max' ,'A04_t2_max','A04_t3_max',
                    # 'A05_t1_mean', 'A05_t2_mean','A05_t3_mean',
                     'A05_t1_max','A05_t2_max','A05_t3_max',
                    # 'A06_t1_mean', 'A06_t2_mean', 'A06_t3_mean',
                    # 'A07_t1_mean', 'A07_t2_mean', 'A07_t3_mean', 
                    ], axis=1) # 5,6,7의 mean만 쓴게 1.191
cat_features = ['area', 'season']
for i in enumerate(cat_features):
    ca = i[1]
    final[ca] = final[ca].astype('category')
# train, test split
train = final[final['year'] != 2016].drop('yyyymmdd', axis=1)
test = final[final['year'] == 2016].drop(['frequency', 'yyyymmdd'], axis=1)

X = train.drop('frequency', axis=1)
y = train['frequency']

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42, stratify=X['month'])

# # 모델링 
# LGBM = LGBMRegressor(objective='mse', verbose=1, metric='rmse', random_state=42, n_jobs=-1, n_estimators = 3000)
# LGBM.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=1, categorical_feature = 'auto')

In [ ]:
# 학습함수 정의 
def objective(trial: Trial):
    param = {
        'objective': 'regression',
        'verbose' : -1,
        'metric' : 'rmse',
        'random_state': 42, 
        'n_jobs': -1,
        'n_estimators' : 3000,
        'num_leaves' : trial.suggest_int('num_leaves', 2, 10, 1),
        'subsample' : trial.suggest_float('subsample', 0.7, 1.0, step=0.05),
        'min_child_samples' : trial.suggest_int('min_child_samples', 2, 150, 2),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 20, 400, 2),
        'reg_alpha' : trial.suggest_float('reg_alpha', 1, 1.5, step = 0.1),
        'reg_lambda' : trial.suggest_float('reg_lambda', 1, 1.5, step = 0.1)
    }
    model = LGBMRegressor(**param, random_seed=42)
    model.fit(X_train, y_train, eval_set=[(X_val,y_val)], categorical_feature = 'auto', verbose=False, early_stopping_rounds=100)
    rmse = RMSE(y_val, model.predict(X_val))

    return rmse

# create study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='LGBMRegressor-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial), n_trials=300)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
LGBM = LGBMRegressor(**study.best_trial.params, random_seed=42, learning_rate=0.05)
LGBM.fit(X, y, verbose=False)
preds = LGBM.predict(test)

year_frequency = final.groupby(['year'])['frequency'].mean()
rate = year_frequency.values[3] / year_frequency.values[2]

submit = pd.read_csv('/content/drive/MyDrive/ML_projects/weather/2-2_검증데이터셋.csv', encoding='cp949')   
submit['frequency'] = preds * rate
submit.to_csv('/content/drive/MyDrive/ML_projects/weather/raw_2/220139.csv', encoding='cp949', index=False)

In [ ]:
# kfold optuna
def objective(trial: Trial, train):
    param = {
        'objective': 'regression',
        'verbose' : -1,
        'metric' : 'rmse',
        'random_state': 42, 
        'n_jobs': -1,
        'n_estimators' : 3000,
        'num_leaves' : trial.suggest_int('num_leaves', 2, 10, 1),
        'subsample' : trial.suggest_float('subsample', 0.7, 1.0, step=0.05),
        'min_child_samples' : trial.suggest_int('min_child_samples', 2, 150, 2),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 20, 400, 2),
        'reg_alpha' : trial.suggest_float('reg_alpha', 1, 1.5, step = 0.1),
        'reg_lambda' : trial.suggest_float('reg_lambda', 1, 1.5, step = 0.1)
    }
    cv_accuracy = []
    cv = KFold(n_splits = 10)
    n_iter = 0
    
    for t,v in cv.split(train, train['frequency']):
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('frequency', axis=1)
        y = train_cv['frequency']

        val_X = val_cv.drop('frequency', axis=1)
        val_y = val_cv['frequency']

        model = LGBMRegressor(**param, random_seed=42)

        model.fit(X, y, categorical_feature = 'auto', eval_set=[(val_X, val_y)], eval_metric = 'rmse', verbose=False, early_stopping_rounds=100) 
        score = RMSE(val_y, model.predict(val_X))

        cv_accuracy.append(score)
        n_iter += 1

    return np.mean(cv_accuracy)

# create study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='LGBMRegressor-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial, train), n_trials=200)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2022-07-23 09:04:08,541] A new study created in memory with name: LGBMRegressor-Hyperparameter-Tuning
[I 2022-07-23 09:04:19,624] Trial 0 finished with value: 1.2100524827855144 and parameters: {'num_leaves': 5, 'subsample': 1.0, 'min_child_samples': 110, 'min_data_in_leaf': 248, 'reg_alpha': 1.0, 'reg_lambda': 1.0}. Best is trial 0 with value: 1.2100524827855144.
[I 2022-07-23 09:04:30,060] Trial 1 finished with value: 1.2244245488269243 and parameters: {'num_leaves': 2, 'subsample': 1.0, 'min_child_samples': 92, 'min_data_in_leaf': 290, 'reg_alpha': 1.0, 'reg_lambda': 1.5}. Best is trial 0 with value: 1.2100524827855144.
[I 2022-07-23 09:04:40,249] Trial 2 finished with value: 1.209275257528875 and parameters: {'num_leaves': 9, 'subsample': 0.75, 'min_child_samples': 28, 'min_data_in_leaf': 90, 'reg_alpha': 1.1, 'reg_lambda': 1.3}. Best is trial 2 with value: 1.209275257528875.
[I 2022-07-23 09:04:51,152] Trial 3 finished with value: 1.2098927317348291 and parameters: {'num_leaves

Best trial : score 1.2089771769871158, 
params {'num_leaves': 10, 'subsample': 0.8999999999999999, 'min_child_samples': 40, 'min_data_in_leaf': 164, 'reg_alpha': 1.3, 'reg_lambda': 1.0}
